In [47]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
from package.logger import Timed, rlog, setup
from package import storage
setup("INFO")

In [49]:
from typing import Any, Optional
import sys

import pandas as pd

import mcr_py
from mcr_py import PyLabel
from package import storage, strtime
from package.logger import Timer, rlog
from package.mcr.data import MCRGeoData
from package.mcr.label import (
    IntermediateLabel,
    McRAPTORLabel,
    McRAPTORLabelWithPath,
    merge_intermediate_bags,
)
from package.mcr.output import OutputFormat
from package.mcr.path import PathManager, PathType
from package.raptor.mcraptor_single import McRaptorSingle
from package.raptor.bag import Bag
from package.mcr.bag import (
    IntermediateBags,
    convert_mc_raptor_bags_to_intermediate_bags,
    convert_mlc_bags_to_intermediate_bags,
)

McRAPTORInputBags = dict[str, list[IntermediateLabel]]


from package.osm import osm
from package.geometa import GeoMeta
import os
from package.mcr.mcr import MCR
from package.minute_city import minute_city

In [50]:
city_id = "Koeln"
stops_path = "../data/cleaned/stops.csv"
osm_path = osm.get_osm_path_from_city_id(city_id)

with Timed.info("Reading stops"):
	other_stops_df = storage.read_gdf(stops_path)

if not os.path.exists(osm_path) and city_id:
	rlog.info("Downloading OSM data")
	osm.download_city(city_id, osm_path)
else:
	rlog.info("Using existing OSM data")

osm_reader = osm.new_osm_reader(osm_path)

geo_meta = GeoMeta.load("../data/geometa.pkl")

with Timed.info("Getting OSM graph"):
	nodes, edges = osm.get_graph_for_city_cropped_to_boundary(osm_reader, geo_meta)

[16:52:02] INFO     Reading stops                                                                   ]8;id=924217;file:///tmp/ipykernel_125110/1014092602.py\1014092602.py]8;;\:]8;id=300993;file:///tmp/ipykernel_125110/1014092602.py#5\5]8;;\

           INFO     Reading stops done (0.11 seconds)                                               ]8;id=719101;file:///tmp/ipykernel_125110/1014092602.py\1014092602.py]8;;\:]8;id=525317;file:///tmp/ipykernel_125110/1014092602.py#5\5]8;;\

           INFO     Using existing OSM data                                                        ]8;id=992896;file:///tmp/ipykernel_125110/1014092602.py\1014092602.py]8;;\:]8;id=39941;file:///tmp/ipykernel_125110/1014092602.py#12\12]8;;\

           INFO     Getting OSM graph                                                              ]8;id=995792;file:///tmp/ipykernel_125110/1014092602.py\1014092602.py]8;;\:]8;id=958782;file:///tmp/ipykernel_125110/1014092602.py#18\18]8;;\

           INFO     Loading OSM network from cache                                                 ]8;id=756518;file:///tmp/ipykernel_125110/1014092602.py\1014092602.py]8;;\:]8;id=847944;file:///tmp/ipykernel_125110/1014092602.py#18\18]8;;\

[16:52:05] INFO     Getting OSM graph done (2.72 seconds)                                          ]8;id=857225;file:///tmp/ipykernel_125110/1014092602.py\1014092602.py]8;;\:]8;id=792736;file:///tmp/ipykernel_125110/1014092602.py#18\18]8;;\

In [51]:
start_time = "08:00:00"
start_node_id = 8734972825

geo_meta_path = "../data/geometa.pkl"
stops_path = "../data/cleaned/stops.csv"
structs_path = "../data/structs.pkl"
bicycle_location_path = "../data/bicycle_locations/2022-11-01_09_01_00.csv"
city_id = "Koeln"

disable_paths=False

output_path = "../data/bags.pkl"

In [52]:
logger = rlog
timer = Timed
geo_meta = GeoMeta.load(geo_meta_path)
geo_data = MCRGeoData(
	stops_path,
	structs_path,
	geo_meta,
	city_id,
	bicycle_location_path=bicycle_location_path,
)

[16:52:06] INFO     Reading stops and geo meta                                                           ]8;id=963285;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=767839;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#41\41]8;;\

           INFO     Reading stops and geo meta done (0.09 seconds)                                       ]8;id=746029;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=599368;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#41\41]8;;\

           INFO     Preparing graphs                                                                     ]8;id=452651;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=340595;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#44\44]8;;\

           INFO     Using existing OSM data                                                              ]8;id=716999;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=36726;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#44\44]8;;\

           INFO     Loading OSM network from cache                                                       ]8;id=651826;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=917037;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#44\44]8;;\

[16:53:07] INFO     Preparing graphs done (1:01 minutes)                                                 ]8;id=820423;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=601336;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#44\44]8;;\

           INFO     Creating graph cache                                                                ]8;id=626266;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=306109;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#144\144]8;;\

[16:53:18] INFO     Creating graph cache done (10.86 seconds)                                           ]8;id=608175;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=507904;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#144\144]8;;\

In [53]:
pois = minute_city.fetch_pois_for_area(geo_meta.boundary, nodes)

In [54]:
geo_data.add_pois_to_mm_graph(pois)
geo_data.add_pois_to_walking_graph(pois)

In [55]:
mcr = MCR(geo_data, disable_paths=True, enable_limit=True)

In [56]:

start_time_in_seconds = strtime.str_time_to_seconds(start_time)

bags_i: dict[int, IntermediateBags] = {}

mcr.logger.debug(f"Starting MCR with config: {mcr.__dict__}")

with mcr.timer.info("Running Dijkstra step"):
	walking_result_bags = mcr_py.run_mlc_with_node_and_time(
		mcr.geo_data.walking_graph_cache,
		mcr.geo_data.walking_node_to_resetted_map[f"W{start_node_id}"],
		start_time_in_seconds,
		disable_paths=mcr.disable_paths,
		enable_limit=mcr.enable_limit,
	)

[16:53:27] INFO     Running Dijkstra step                                                              ]8;id=913457;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=126738;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           WARNING  94 labels were discarded because they exceeded the limit                             ]8;id=662103;file:///home/moritz/dev/uni/mlc/src/mlc.rs\mlc.rs]8;;\:]8;id=584350;file:///home/moritz/dev/uni/mlc/src/mlc.rs#307\307]8;;\

           INFO     Running Dijkstra step done (0.09 seconds)                                          ]8;id=800406;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=903190;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

In [57]:
with mcr.timer.info("Extracting dijkstra step bags"):
	walking_result_bags = mcr.convert_walking_bags(
		walking_result_bags,
	)
	bags_i[0] = walking_result_bags

	n_osm_nodes = len(walking_result_bags)
	mcr.logger.debug(f"Found {n_osm_nodes} osm nodes in walking step")

           INFO     Extracting dijkstra step bags                                                      ]8;id=134936;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=184691;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Extracting dijkstra step bags done (0.02 seconds)                                  ]8;id=922310;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=320041;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

In [86]:
test = pd.DataFrame(
    [(label.path, label) for bag in walking_result_bags.values() for label in bag],
    columns=["path", "label"],
)
len(test[test.path.apply(lambda x: len(x) > 1)])

0

In [93]:
test[test.path.astype(str) != '[]'].label.iloc[0]

IntermediateLabel(values=[28800, 0], hidden_values=[0, 0], path=[66346], node_id=8734972825)

In [61]:
i = 1
mcr.logger.info(f"Running iteration {i}")
offset = i * 2 - 1

[16:55:05] INFO     Running iteration 1                                                             ]8;id=917957;file:///tmp/ipykernel_125110/4269777464.py\4269777464.py]8;;\:]8;id=544594;file:///tmp/ipykernel_125110/4269777464.py#2\2]8;;\

In [82]:
bicycle_result_bags = mcr.run_bicycle_step(walking_result_bags, offset)

[autoreload of mcr_py failed: Traceback (most recent call last):
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/mcr_py/__init__.py", line 3, in <module>
    __doc__ = mcr_py.__doc__
NameError: name 'mcr_py' is not defined
]


[17:08:10] INFO     Preparing input for bicycle step                                                   ]8;id=105291;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=279453;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           WARNING  No bicycles reached with walking step                                              ]8;id=649234;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=102446;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Preparing input for bicycle step done (0.01 seconds)                               ]8;id=967626;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=53122;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

In [83]:
with mcr.timer.info("Preparing input for MCRAPTOR step"):
	mc_raptor_bags = mcr.prepare_mcraptor_step_input(walking_result_bags)

[17:08:13] INFO     Preparing input for MCRAPTOR step                                                  ]8;id=662071;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=659062;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

[66346] 647


           INFO     Preparing input for MCRAPTOR step done (0.01 seconds)                              ]8;id=364086;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=103183;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

In [80]:
mc_raptor_bags

{'641': {Label(29555)},
 '640': {Label(29795)},
 '649': {Label(29851)},
 '650': {Label(29690)},
 '817': {Label(29357)},
 '645': {Label(29789)},
 '644': {Label(29923)},
 '647': {McRAPTORLabelWithPath(time=28800, cost=0, n_stops=0, path=[66346])},
 '812': {Label(29171)},
 '813': {Label(29660)},
 '648': {Label(29544)},
 '842': {Label(29902)}}

In [ ]:
with mcr.timer.info("Running MCRAPTOR step"):
	mc_raptor = McRaptorSingle(
		mcr.geo_data.structs_dict,
		default_transfer_time=60,
		label_class=(
			McRAPTORLabel if mcr.disable_paths else McRAPTORLabelWithPath
		),
	)
	mc_raptor_result_bags = mc_raptor.run(mc_raptor_bags)  # type: ignore

with mcr.timer.info("Extracting MCRAPTOR step bags"):
	mc_raptor_result_bags = mcr.convert_mc_raptor_bags(
		mc_raptor_result_bags, path_index_offset=offset
	)
	if len(mc_raptor_result_bags) == 0:
		mcr.logger.warn("No MCRAPTOR bags found")

In [78]:
mc_raptor_result_bags = mcr.run_mc_raptor_step(walking_result_bags, offset)

[autoreload of mcr_py failed: Traceback (most recent call last):
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/mcr_py/__init__.py", line 3, in <module>
    __doc__ = mcr_py.__doc__
NameError: name 'mcr_py' is not defined
]


[17:05:35] INFO     Preparing input for MCRAPTOR step                                                  ]8;id=922552;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=183999;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Preparing input for MCRAPTOR step done (0.01 seconds)                              ]8;id=199182;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=372583;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Running MCRAPTOR step                                                              ]8;id=867587;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=262555;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           WARNING  Added 824 missing stops to bags (98.56%)                                           ]8;id=931496;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=883643;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

[McRAPTORLabelWithPath(time=29040, cost=220, n_stops=1, path=[66346, 'STOP_647', 'TRIP_9-151-001-467.2.21:064500-47-2_B7E643E3-5616-4606-9BDE-B03101412D17'])]
[Label(29880), McRAPTORLabelWithPath(time=29100, cost=220, n_stops=2, path=[66346, 'STOP_647', 'TRIP_9-151-001-467.2.21:064500-47-2_B7E643E3-5616-4606-9BDE-B03101412D17'])]
[McRAPTORLabelWithPath(time=29160, cost=220, n_stops=1, path=[66346, 'STOP_647', 'TRIP_119-151-001-814.2.22:080200-47-2_C26846EF-6A43-4C62-BC00-B0310141273C'])]
[McRAPTORLabelWithPath(time=29220, cost=220, n_stops=2, path=[66346, 'STOP_647', 'TRIP_119-151-001-814.2.22:080200-47-2_C26846EF-6A43-4C62-BC00-B0310141273C']), Label(30420)]
[McRAPTORLabelWithPath(time=29280, cost=220, n_stops=3, path=[66346, 'STOP_647', 'TRIP_119-151-001-814.2.22:080200-47-2_C26846EF-6A43-4C62-BC00-B0310141273C']), Label(30480)]
[McRAPTORLabelWithPath(time=29340, cost=220, n_stops=4, path=[66346, 'STOP_647', 'TRIP_119-151-001-814.2.22:080200-47-2_C26846EF-6A43-4C62-BC00-B0310141273C'

           INFO     Running MCRAPTOR step done (0.03 seconds)                                          ]8;id=933306;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=802415;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Extracting MCRAPTOR step bags                                                      ]8;id=802000;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=751091;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Extracting MCRAPTOR step bags failed (0.01 seconds)                                ]8;id=977370;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=302703;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

ValueError: Expected McRAPTORLabel, got <class 'package.mcr.label.McRAPTORLabel'> instead

In [70]:
test = pd.DataFrame(
    [(label.path, label) for bag in mc_raptor_result_bags.values() for label in bag],
    columns=["path", "label"],
)
len(test[test.path.apply(lambda x: len(x) > 1)])

[autoreload of mcr_py failed: Traceback (most recent call last):
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/mcr_py/__init__.py", line 3, in <module>
    __doc__ = mcr_py.__doc__
NameError: name 'mcr_py' is not defined
]


37

In [19]:
with mcr.timer.info("Merging bags"):
	combined_bags = mcr.merge_bags(
		bicycle_result_bags,
		mc_raptor_result_bags,
	)

[16:44:28] INFO     Merging bags                                                                       ]8;id=120878;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=304956;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Merging bags done (0.01 seconds)                                                   ]8;id=417654;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=859141;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

In [ ]:
with mcr.timer.info("Preparing input for walking step"):
	walking_bags = mcr.prepare_walking_step_input(combined_bags)

In [45]:
test = pd.DataFrame(
    [(label.path, label) for bag in walking_result_bags.values() for label in bag],
    columns=["path", "label"],
)
len(test[test.path.apply(lambda x: len(x) > 1)])

37

In [21]:

with mcr.timer.info("Running walking step"):
	walking_result_bags = mcr_py.run_mlc_with_bags(
		mcr.geo_data.walking_graph_cache,
		walking_bags,
		disable_paths=mcr.disable_paths,
		enable_limit=mcr.enable_limit,
	)

[16:44:39] INFO     Preparing input for walking step                                                   ]8;id=575204;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=325872;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Preparing input for walking step done (0.22 seconds)                               ]8;id=62469;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=169291;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Running walking step                                                               ]8;id=851005;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=728597;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

thread '<unnamed>' panicked at 'called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError("'str' object cannot be interpreted as an integer"), traceback: None }', src/rs/mlc_adapter.rs:142:18
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


           INFO     Running walking step failed (0.01 seconds)                                         ]8;id=403815;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=883290;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

PanicException: called `Result::unwrap()` on an `Err` value: PyErr { type: <class 'TypeError'>, value: TypeError("'str' object cannot be interpreted as an integer"), traceback: None }

In [ ]:
with mcr.timer.info("Extracting walking step bags"):
	walking_result_bags = mcr.convert_walking_bags(
		walking_result_bags, path_index_offset=offset + 1
	)

bags_i[i] = walking_result_bags

with mcr.timer.info("Saving bags"):
mcr.save_bags(bags_i, output_path)


In [ ]:
stops_path = "../data/cleaned/stops.csv"
city_id = "Koeln"
osm_path = ""
structs = "../data/structs.pkl"
bicycle_location_path = "../data/bicycle_locations/2022-11-01_09_01_00.csv"
geo_meta_path = "../data/geometa.pkl"

mcr_geo_data = MCRGeoData(
    stops_path,
    structs,
    geo_meta_path,
    city_id,
    bicycle_location_path=bicycle_location_path,
)


[09:16:17] INFO     Reading stops and geo meta                                                           ]8;id=523592;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=505202;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#40\40]8;;\

           INFO     Reading stops and geo meta done (0.10 seconds)                                       ]8;id=996382;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=670565;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#40\40]8;;\

           INFO     Preparing graphs                                                                     ]8;id=732994;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=873269;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#44\44]8;;\

           INFO     Using existing OSM data                                                              ]8;id=98038;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=541345;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#44\44]8;;\

           INFO     Loading OSM network from cache                                                       ]8;id=569089;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=326929;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#44\44]8;;\

[09:16:57] INFO     Reading bicycle locations                                                            ]8;id=766786;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=245330;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#60\60]8;;\

           INFO     Reading bicycle locations done (0.38 seconds)                                        ]8;id=645157;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=603030;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#60\60]8;;\

[09:17:20] INFO     Preparing graphs done (1:02 minutes)                                                 ]8;id=730234;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=417376;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#44\44]8;;\

           INFO     Creating graph cache                                                                ]8;id=132980;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=330086;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#147\147]8;;\

[09:17:30] INFO     Creating graph cache done (10.46 seconds)                                           ]8;id=27891;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py\data.py]8;;\:]8;id=766126;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/data.py#147\147]8;;\

In [ ]:
mcr_geo_data.add_pois_to_mm_graph(pois)

In [ ]:
mcr = MCR(mcr_geo_data, disable_paths=False, enable_limit=False)
mcr.run(1638242852, "08:00:00", 2, output_path="../data/bags.pkl")

[09:19:20] INFO     Running Dijkstra step                                                              ]8;id=378213;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=746871;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

[09:19:26] INFO     Running Dijkstra step done (5.60 seconds)                                          ]8;id=670575;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=115860;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Extracting dijkstra step bags                                                      ]8;id=127319;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=930482;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

[09:19:38] INFO     Extracting dijkstra step bags done (12.29 seconds)                                 ]8;id=416563;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=956125;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

           INFO     Running iteration 1                                                                   ]8;id=248679;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/mcr.py\mcr.py]8;;\:]8;id=359458;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/mcr.py#94\94]8;;\

           INFO     Preparing input for bicycle step                                                   ]8;id=170041;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=512117;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

[09:19:39] INFO     Preparing input for bicycle step done (0.77 seconds)                               ]8;id=757033;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py\logger.py]8;;\:]8;id=355630;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/logger.py#37\37]8;;\

: 